Microsoft Research Paraphrasing Corpus

In [1]:
# Importing required libraries

!pip install transformers
!pip install datasets

import tensorflow as tf
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import DataCollatorWithPadding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [2]:
mydataset = load_dataset("glue", "mrpc")
mydataset

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [3]:
checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [4]:
def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = mydataset.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [6]:
# Now lets prepare the tokenized dataset for training, as we left the padding earlier in tokenize_function, now put collatefunction with the tokenized_dataset to pad the dataset and convert it format that keras models understand


tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)


model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model.fit(tf_train_dataset, epochs=10, batch_size=8, validation_data=tf_validation_dataset)

Epoch 1/10
459/459 [==============================] - 132s 189ms/step - loss: 0.6557 - accuracy: 0.6557 - val_loss: 0.6344 - val_accuracy: 0.6838
Epoch 2/10
459/459 [==============================] - 77s 168ms/step - loss: 0.6850 - accuracy: 0.6404 - val_loss: 0.6761 - val_accuracy: 0.6838
Epoch 3/10
459/459 [==============================] - 76s 166ms/step - loss: 0.6999 - accuracy: 0.6210 - val_loss: 0.6807 - val_accuracy: 0.6838
Epoch 4/10
459/459 [==============================] - 77s 167ms/step - loss: 0.6946 - accuracy: 0.6276 - val_loss: 0.7075 - val_accuracy: 0.3162
Epoch 5/10
459/459 [==============================] - 77s 167ms/step - loss: 0.6736 - accuracy: 0.6363 - val_loss: 0.6261 - val_accuracy: 0.6838
Epoch 6/10
459/459 [==============================] - 77s 167ms/step - loss: 0.6607 - accuracy: 0.6538 - val_loss: 0.6268 - val_accuracy: 0.6838
Epoch 7/10
459/459 [==============================] - 80s 173ms/step - loss: 0.6645 - accuracy: 0.6489 - val_loss: 0.6252 - val_a

In [8]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [9]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=10)

Epoch 1/10
459/459 [==============================] - 129s 183ms/step - loss: 0.5097 - accuracy: 0.7514 - val_loss: 0.3731 - val_accuracy: 0.8431
Epoch 2/10
459/459 [==============================] - 76s 165ms/step - loss: 0.2300 - accuracy: 0.9117 - val_loss: 0.3893 - val_accuracy: 0.8456
Epoch 3/10
459/459 [==============================] - 76s 166ms/step - loss: 0.0708 - accuracy: 0.9787 - val_loss: 0.4502 - val_accuracy: 0.8529
Epoch 4/10
459/459 [==============================] - 79s 173ms/step - loss: 0.0320 - accuracy: 0.9937 - val_loss: 0.4502 - val_accuracy: 0.8529
Epoch 5/10
459/459 [==============================] - 77s 169ms/step - loss: 0.0301 - accuracy: 0.9932 - val_loss: 0.4502 - val_accuracy: 0.8529
Epoch 6/10
459/459 [==============================] - 77s 168ms/step - loss: 0.0323 - accuracy: 0.9935 - val_loss: 0.4502 - val_accuracy: 0.8529
Epoch 7/10
459/459 [==============================] - 78s 169ms/step - loss: 0.0311 - accuracy: 0.9935 - val_loss: 0.4502 - val_a